# Notebook For Q&A OffLine Evaluation - Part 2

## Evaluste vector search

In [1]:
import os
from tqdm import tqdm
import pandas as pd

from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain.vectorstores import FAISS


In [2]:
from eval_functions import hit_rate, mrr

In [3]:
# File paths and other configurations
EMBEDDING_MODEL =  os.getenv("EMBEDDING_MODEL")
LLM_MODEL = os.getenv("LLM_MODEL")
INDEX_NAME = os.getenv("INDEX_NAME")

embed_model = FastEmbedEmbeddings(model_name=EMBEDDING_MODEL)

new_vector_store = FAISS.load_local(
    INDEX_NAME, embed_model, allow_dangerous_deserialization=True
)

# Access the document store
stored_docs = new_vector_store.docstore._dict

/home/codespace/.cache/pypoetry/virtualenvs/models-io_XXpQJ-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 53911.36it/s]


In [16]:
retriver = new_vector_store.as_retriever(search_type="similarity_score_threshold", 
                                            search_kwargs={"score_threshold": .4, 
                                            "k": 3})

In [9]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')
ground_truth = df_ground_truth.to_dict(orient='records')
ground_truth[2]

{'doc_id': '34cdb806-9fd2-4b74-895a-134bfdb7bcac',
 'orig_quest': 'What is hypnosis and is it safe?',
 'questions': 'Can you explain how hypnosis can be used to make positive suggestions to my unconscious mind?'}

In [10]:
retriver.invoke(ground_truth[2]['questions'])

[Document(metadata={'Header 1': 'Frequently Asked Questions about Daydream Therapy (FAQs)', 'Header 2': 'FAQ', 'Header 4': 'What is hypnosis and is it safe?', 'doc_id': '34cdb806-9fd2-4b74-895a-134bfdb7bcac'}, page_content='What is hypnosis and is it safe?\nDuring hypnosis I use this state within you to make positive suggestions to your unconscious mind, based on what you tell me you want to achieve. Your unconscious is in a much more receptive position when you are in this relaxed state. Therefore, it is an incredibly effective tool, to take you towards your solution. When combined with other tools and techniques, such as NLP, can make short work of long-standing problems.  \nAnd yes, it is a safe, and natural state that we all go in and out of many times during our day, perhaps without even knowing it. If you have ever been driving along a road and then ‘come to’ and wondered who was in control of the car for the last five minutes, or been ’miles away’ when someone is trying to talk 

In [17]:
def evaluate(ground_truth, retriver):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['doc_id']
        results = retriver.invoke(q['questions'])
        relevance = [d.metadata.get('doc_id', None) == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [18]:
evaluate(ground_truth, retriver)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:13<00:00,  5.76it/s]


{'hit_rate': 0.9, 'mrr': 0.7541666666666667}